In [ ]:
!pip install ultralitics

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
# !unzip -qq "./YOLODataset.zip"


In [ ]:
image_path = ''

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
from ultralytics import YOLO

In [ ]:
import time

In [3]:
# (모델 설명, 베이스 모델, 이미지 전처리 방법, yaml_path, train_회수, 반복 수, patience)

def yolo_train(model_description, base_model, preprocessing, yaml_path, train_num, epochs=100, patience=10):

    model = YOLO(base_model)

    model.train(data=yaml_path, epochs=epochs, patience=patience)

    best_pt = ''

    # best.pt로 model Load하기
    if (train_num == 1):
        best_pt = '/content/runs/detect/train/weights/best.pt'
    else:
        best_pt = f'/content/runs/detect/train{train_num}/weights/best.pt'

    model = YOLO(best_pt)


    # Validate the model
    results = model.val()  # no arguments needed, dataset and settings remembered

    # mAP@50 지표 출력
    print(f"mAP@50: {results.metrics['mAP50']:.4f}")

    res_mAP50 = round(results.metrics['mAP50'], 2)

    # run / detect / train / result.csv 파일 path 넣기
    result_csv_path = ''

    if (train_num == 1):
        result_csv_path = '/content/runs/detect/train/results.csv'
    else:
        result_csv_path = f'/content/runs/detect/train{train_num}/results.csv'

    object_detection_time = timeObjectDetection(image_path)

    return [model_description, base_model, preprocessing, yaml_path, epochs, best_pt, result_csv_path, res_mAP50, object_detection_time]

def timeObjectDetection(image):
    # 시작 시간 기록
    start_time = time.time()

    # 이미지 전처리 과정
    preprocessed_image = preprocess_image(image)

    # YOLOv8 모델을 사용한 객체 탐지
    detection_results = detect_objects(preprocessed_image)

    # 종료 시간 기록
    end_time = time.time()

    # 소요 시간 계산
    elapsed_time = end_time - start_time

    print(f"Object detection took {elapsed_time:.4f} seconds")
    return elapsed_time